To run this notebook using the project venv, you will need to install the jupyter into it. After activate your venv, run the commands below:

```bash
$ (venv) pip3 install jupyter
$ (venv) ipython kernel install --name "hyperheuristics-venv" --user
```

Next, just restart your Jupyter and change the kernel that your Jupyter notebook is using (if you are running it into VSCode, the kernel is located in the top right corner). 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

import numpy as np
import yaml
import os

from typing import Text, Dict

from src.hyperHeuristics import HyperHeuristic
from utils.generator.knapsackInstanceGenerator import generateKnapsackInstance

In [3]:
# Data path
DATA_PATH = "../data"
os.makedirs(DATA_PATH, exist_ok = True)

In [9]:
config = yaml.load(open('experimentConfig.yaml'), yaml.Loader)
stages = config['stages']
firstStages = np.array(stages['firstStage'])
secondStages = np.array(stages['secondStage'])
thirdStages = np.array(stages['thirdStage'])
params = config['params']

In [5]:
def getRandomStageParams(stage: Text) -> Dict:
    stageParams = params[stage]
    
    sortedParams = {param: values[int(np.random.rand() * len(values))] for param, values in stageParams.items()}
    return sortedParams

In [6]:
def getRandomThreeStagesPipeline(
        firstStageCandidates: np.array, 
        secondStageCandidates: np.array, 
        thirdStageCandidates: np.array
    ) -> HyperHeuristic:
    
    hyperheuristic = HyperHeuristic()
    lowLevelFunctions = hyperheuristic.getAvailableLowLevelHeuristics()

    # Sort first stage
    firstStage = np.random.choice(firstStageCandidates, 1)[0]
    firstStageParams = getRandomStageParams(firstStage)

    firstStageCandidates = firstStageCandidates[firstStageCandidates != firstStage]
    thirdStageCandidates = thirdStageCandidates[thirdStageCandidates != firstStage]

    # Sort second stage
    secondStage = np.random.choice(secondStageCandidates, 1)[0]
    secondStageParams = getRandomStageParams(secondStage)

    thirdStageCandidates = thirdStageCandidates[thirdStageCandidates != secondStage]

    # Sort third stage
    thirdStage = np.random.choice(thirdStageCandidates, 1)[0]
    thirdStageParams = getRandomStageParams(thirdStage)

    stages = [
        {
            "name": firstStage,
            "callback": lowLevelFunctions[firstStage],
            "inputSchema": ['problemInstance'],
            "outputSchema": ['problemInstance', 'solution'],
            "params": firstStageParams
        },
        {
            "name": secondStage,
            "callback": lowLevelFunctions[secondStage],
            "inputSchema": ['problemInstance','solution'],
            "outputSchema": ['problemInstance', 'solution'],
            "params": secondStageParams
        },
        {
            "name": thirdStage,
            "callback": lowLevelFunctions[thirdStage],
            "inputSchema": ['problemInstance','solution'],
            "outputSchema": ['problemInstance', 'solution'],
            "params": thirdStageParams
        },
    ]

    hyperheuristic.addHyperHeuristicComponentsFromDict(stages)

    print(" -> ".join(hyperheuristic.getStagesNames()))

    return hyperheuristic

In [7]:
def storeCompositionData(stage):
    name = stage.name
    params = stage.params
    solution = stage.output['problemInstance'].solution
    solutionFO = stage.output['problemInstance'].objective(isMinimizing = False)
    processTime = stage.processTime
    print(name)
    print(params)
    print(solution)
    print(solutionFO)
    print(processTime)
    print("")

In [ ]:
for i in range(100): 
    instancePath = f"{DATA_PATH}/instance_{i}.txt"
    instance = generateKnapsackInstance(10,50,2,7,100,900,10,90,instancePath)
    print("Best FO for instance %d: %d"%(i, instance.objective(isMinimizing = False)))
    for j in range(10):
        composition = getRandomThreeStagesPipeline(firstStages, secondStages, thirdStages)
        composition.getSolution(instance, storeCompositionData)